In [ ]:
# %% [markdown]
"""
# Proyecto: Predictor de Calidad de Vino 🍷
**Fase de Google Colab: Exploración, Preprocesamiento y Entrenamiento del Modelo**

**Objetivo**: Cargar y explorar el dataset, preprocesar los datos (manejo de ausentes y escalado),
entrenar un modelo de Machine Learning para predecir la calidad del vino (regresión),
y guardar el modelo junto con los preprocesadores para su uso en la aplicación GUI.
"""

# %% [markdown]
"""
## Paso 1: Configuración Inicial y Carga de Librerías
"""

# %%
# Librerías esenciales para manipulación de datos y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn para preprocesamiento y modelado
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Para guardar y cargar objetos de Python (modelos, transformadores)
import joblib

# %% [markdown]
"""
## Paso 2: Carga y Exploración de Datos (EDA)
"""

# %%
# 2.1 Cargar el Wine Quality Dataset
file_path_drive = '/content/winequality-red.csv' 
df = None 

try:
    df = pd.read_csv(file_path_drive, sep=';')
    print(f"Dataset cargado exitosamente desde '{file_path_drive}' con separador ';'.")
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta '{file_path_drive}'.")
    print("Por favor, asegúrate de que el archivo CSV esté en tu Google Drive y la ruta sea correcta.")
    print("Creando un DataFrame de ejemplo para continuar la ejecución.")
    # Crear un DataFrame de ejemplo para que el resto del código funcione
    data = {
        'fixed acidity': np.random.rand(100) * 5 + 5,
        'volatile acidity': np.random.rand(100) * 0.5 + 0.2,
        'citric acid': np.random.rand(100) * 0.5,
        'residual sugar': np.random.rand(100) * 5 + 1,
        'chlorides': np.random.rand(100) * 0.1 + 0.03,
        'free sulfur dioxide': np.random.rand(100) * 50 + 10,
        'total sulfur dioxide': np.random.rand(100) * 100 + 50,
        'density': np.random.rand(100) * 0.005 + 0.995,
        'pH': np.random.rand(100) * 0.5 + 3.0,
        'sulphates': np.random.rand(100) * 0.5 + 0.5,
        'alcohol': np.random.rand(100) * 5 + 9,
        'quality': np.random.randint(3, 9, 100)
    }
    df = pd.DataFrame(data)
    print("¡Advertencia! Se ha cargado un DataFrame de ejemplo. Para el proyecto real, debes cargar el dataset de vino.")
except pd.errors.ParserError:
    print(f"Error de parsing al cargar '{file_path_drive}'. Probablemente el separador no es ';'.")
    print("Intentando cargar con ',' como separador.")
    try:
        df = pd.read_csv(file_path_drive, sep=',')
        print(f"Dataset cargado exitosamente desde '{file_path_drive}' con separador ','.")
    except Exception as e:
        print(f"Error al cargar el dataset con ambos separadores. Por favor, verifica la ruta y el archivo: {e}")
        print("Creando un DataFrame de ejemplo como último recurso.")
        data = { # DataFrame de ejemplo (mismo que antes)
            'fixed acidity': np.random.rand(100) * 5 + 5, 'volatile acidity': np.random.rand(100) * 0.5 + 0.2,
            'citric acid': np.random.rand(100) * 0.5, 'residual sugar': np.random.rand(100) * 5 + 1,
            'chlorides': np.random.rand(100) * 0.1 + 0.03, 'free sulfur dioxide': np.random.rand(100) * 50 + 10,
            'total sulfur dioxide': np.random.rand(100) * 100 + 50, 'density': np.random.rand(100) * 0.005 + 0.995,
            'pH': np.random.rand(100) * 0.5 + 3.0, 'sulphates': np.random.rand(100) * 0.5 + 0.5,
            'alcohol': np.random.rand(100) * 5 + 9, 'quality': np.random.randint(3, 9, 100)
        }
        df = pd.DataFrame(data)


# 2.2 Análisis exploratorio inicial del dataset
print("\n--- Vista Previa del Dataset (Primeras 5 filas) ---")

print(df.head())

print("\n--- Columnas del DataFrame después de la carga ---")
print(df.columns)

print("\n--- Información General del Dataset ---")
df.info()

print("\n--- Estadísticas Descriptivas de las Columnas Numéricas ---")
print(df.describe())

print("\n--- Verificación de Valores Ausentes en el Dataset Original ---")
print(df.isnull().sum())

if df.isnull().sum().sum() == 0:
    print("\nConfirmación: El dataset original no contiene valores ausentes.")
else:
    print("\nAdvertencia: El dataset original contiene valores ausentes. Esto podría requerir preprocesamiento adicional si no es intencionado.")


print("\n--- Distribución de la Variable Objetivo 'quality' ---")
plt.figure(figsize=(8, 5))
sns.countplot(x='quality', data=df)
plt.title('Distribución de la Calidad del Vino')
plt.xlabel('Calidad')
plt.ylabel('Número de Vinos')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# %% [markdown]
"""
## Paso 3: Preprocesamiento de Datos para el Modelo
"""

# %%
# 3.1 Separar características (X) y la variable objetivo (y)
X = df.drop('quality', axis=1) # Todas las columnas excepto 'quality' son características de entrada
y = df['quality'] # 'quality' es la variable que queremos predecir

print(f"\nDimensiones de X (características): {X.shape}")
print(f"Dimensiones de y (variable objetivo): {y.shape}")

# 3.2 Dividir el dataset en conjuntos de entrenamiento y prueba
# Usaremos un 80% para entrenamiento y un 20% para prueba.
# random_state asegura que la división sea reproducible.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDimensiones del conjunto de entrenamiento X_train: {X_train.shape}")
print(f"Dimensiones del conjunto de prueba X_test: {X_test.shape}")
print(f"Dimensiones del objetivo de entrenamiento y_train: {y_train.shape}")
print(f"Dimensiones del objetivo de prueba y_test: {y_test.shape}")

# 3.3 Imputación de Valores Ausentes (para la inferencia futura en la GUI)
# Aunque el dataset de entrenamiento no tiene ausentes, necesitamos un imputer entrenado
# para manejar los valores ausentes que el usuario podría no ingresar en la GUI.
# Usaremos KNNImputer para una imputación más robusta.
# n_neighbors: número de vecinos a considerar para la imputación. 5 es un valor común.
imputer = KNNImputer(n_neighbors=5)

# Ajustar el imputer SOLO al conjunto de entrenamiento.
# Esto asegura que la imputación se base solo en los datos "conocidos" por el modelo.
imputer.fit(X_train)

# Transformar los conjuntos de entrenamiento y prueba usando el imputer (aunque no tengan ausentes,
# esto es para completar la secuencia si el imputer se usara en un pipeline).
# Para este flujo, es más importante guardar el imputer ajustado.
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)


# Guardar el imputer entrenado. Este archivo se cargará en la aplicación GUI.
imputer_filename = '/content/wine_quality_imputer.joblib'
joblib.dump(imputer, imputer_filename)
print(f"\nImputer (KNNImputer) guardado en Google Drive como '{imputer_filename}'.")


# 3.4 Escalado de Características
# Creamos un escalador para normalizar los datos (media 0, varianza 1).
# Esto es una buena práctica para muchos modelos de ML, aunque Random Forest es menos sensible.
scaler = StandardScaler()

# Ajustamos el escalador SOLO en el conjunto de entrenamiento y luego lo transformamos.
# Es fundamental para evitar la fuga de datos (data leakage) del conjunto de prueba.
X_train_scaled = scaler.fit_transform(X_train_imputed) # Aplicamos escalado sobre datos ya imputados
X_test_scaled = scaler.transform(X_test_imputed) # Transformamos test con el scaler ajustado en train

print("\nCaracterísticas escaladas usando StandardScaler.")
print(f"Primeras 5 filas de X_train_scaled (escalado):\n{X_train_scaled[:5]}")

# Guardar el scaler entrenado. Este archivo también se cargará en la aplicación GUI.
scaler_filename = '/content/wine_quality_scaler.joblib'
joblib.dump(scaler, scaler_filename)
print(f"\nScaler (StandardScaler) guardado en Google Drive como '{scaler_filename}'.")

# %% [markdown]
"""
## Paso 4: Desarrollo del Modelo de Machine Learning
"""

# %%
print("\n--- Iniciando el Desarrollo del Modelo de Machine Learning ---")

# 4.1 Elección y Configuración del Algoritmo de ML
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
# n_estimators: número de árboles. Más árboles suelen dar mejor rendimiento.
# random_state: para reproducibilidad.
# n_jobs=-1: usa todos los cores del CPU disponibles para acelerar el entrenamiento.
print("\nModelo seleccionado: Random Forest Regressor")
print(f"Parámetros del modelo: {model.get_params()}")

# 4.2 Entrenar el Modelo
# Entrenamos el modelo con los datos de entrenamiento ya preprocesados (imputados y escalados).
print("\nEntrenando el modelo...")
model.fit(X_train_scaled, y_train)
print("¡Modelo entrenado exitosamente!")

# 4.3 Evaluación del Rendimiento del Modelo
# Realizamos predicciones sobre el conjunto de prueba escalado.
y_pred = model.predict(X_test_scaled)

# Calculamos métricas de evaluación para modelos de regresión.
# Mean Squared Error (MSE): Mide el error promedio de las predicciones al cuadrado.
# Un valor más bajo indica que las predicciones están más cerca de los valores reales.
mse = mean_squared_error(y_test, y_pred)
print(f"\nError Cuadrático Medio (MSE) en el conjunto de prueba: {mse:.4f}")

# Root Mean Squared Error (RMSE): Es la raíz cuadrada del MSE, lo que lo hace más interpretable
# al estar en la misma unidad que la variable objetivo (calidad).
rmse = np.sqrt(mse)
print(f"Raíz del Error Cuadrático Medio (RMSE) en el conjunto de prueba: {rmse:.4f}")


# R-squared (R2 Score): Mide la proporción de la varianza en la variable objetivo que es
# predecible a partir de las características de entrada.
# Un valor más cercano a 1 indica un mejor ajuste del modelo.
r2 = r2_score(y_test, y_pred)
print(f"Coeficiente de Determinación (R2 Score) en el conjunto de prueba: {r2:.4f}")

# Visualización de las predicciones vs. valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
# Línea de referencia ideal: donde la predicción es igual al valor real
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title('Predicciones del Modelo vs. Valores Reales de Calidad')
plt.xlabel('Calidad Real')
plt.ylabel('Calidad Predicha')
plt.grid(True)
plt.show()

# 4.4 Guardar el Modelo Entrenado
# Es esencial guardar el modelo entrenado para poder cargarlo y usarlo en la aplicación GUI
# sin tener que reentrenarlo cada vez.
# Usamos joblib porque es eficiente para guardar y cargar objetos de scikit-learn.
model_filename = '/content/wine_quality_model.joblib'
joblib.dump(model, model_filename)
print(f"\nModelo entrenado guardado exitosamente en: {model_filename}")

print("\n--- Desarrollo del Modelo de Machine Learning Completado ---")